In [ ]:
import pandas as pd
from pathlib import Path
import glob

In [ ]:
def prep_keys(df):
    df.drop('T_REC', axis=1, inplace=True)
    df.drop('WAVE_STR', axis=1, inplace=True)
    df.drop('PIXLUNIT', axis=1, inplace=True)
    date_obs_c = df['DATE__OBS'].apply(lambda dobs: fix_date_obs(dobs))
    df['DATE_OBS_c'] = date_obs_c
    df['T_OBS'] = pd.to_datetime(df['T_OBS'], utc=True)
    df['DATE'] = pd.to_datetime(df['DATE'], utc=True)
    df.set_index('T_OBS', inplace=True)
    return 

def fix_date_obs(dobs):
    sec = pd.Timedelta(int(dobs[-6:-4]), unit='s')
    if sec < pd.Timedelta(60, unit='s'):
        return pd.to_datetime(dobs, utc=True)
    print(dobs)
    trunc_date = pd.to_datetime(dobs[0:-7], utc=True)
    hundredth_sec =  pd.Timedelta(int(dobs[-3:-1])/100, unit='s')
    corrected_date = trunc_date + sec + hundredth_sec
    print('corrected: ', corrected_date)
    return corrected_date

In [ ]:
files = sorted(glob.glob('/mnt/data2/SDO/AIA/keys*.txt'))
fpaths = [Path(f) for f in files]
fpaths

In [ ]:
for f in fpaths:
    print(f)
    df = pd.read_csv(f, delimiter='\t')
    prep_keys(df)
    df.to_parquet(f.with_suffix('.parquet'))